# O Problema de Atribuição de Enfermeiros
-  [Descrever o problema de negócios](#Describe-the-business-problem)
*  [Usar a otimização de decisões](#Use-decision-optimization)
    *  [Passo 1: Importar a biblioteca](#Step-1:-Import-the-library)
    -  [Passo 2: Modelar os dados](#Step-2:-Model-the-data)
    *  [Passo 3: Preparar os dados](#Step-3:-Prepare-the-data)
    -  [Passo 4: Configurar o modelo prescritivo](#Step-4:-Set-up-the-prescriptive-model)
        * [Definir as variáveis de decisão](#Define-the-decision-variables)
        * [Expressar as restrições de negócios](#Express-the-business-constraints)
        * [Expressar o objetivo](#Express-the-objective)
        * [Resolver com Otimização de Decisões](#Solve-with-Decision-Optimization)
    *  [Passo 5: Investigar a solução e executar uma análise de exemplo](#Step-5:-Investigate-the-solution-and-then-run-an-example-analysis)
*  [Resumo](#Summary)

## Descrever o problema de negócios

Este notebook descreve como usar o CPLEX Modeling para Python junto com *pandas* para gerenciar a atribuição de enfermeiros aos turnos em um hospital.

Os enfermeiros devem ser atribuídos aos turnos do hospital de acordo com várias restrições de habilidades e de pessoal.

O objetivo do modelo é encontrar um equilíbrio eficiente entre diferentes metas:

* minimizar o custo total do plano e
* atribuir os turnos de forma mais justa possível.

## Verificando os requisitos mínimos

Este notebook utiliza algumas funcionalidades do pandas que estão disponíveis na versão 0.17.1 ou superior.

In [1]:
import pip
REQUIRED_MINIMUM_PANDAS_VERSION = '0.17.1'
try:
    import pandas as pd
    assert pd.__version__ >= REQUIRED_MINIMUM_PANDAS_VERSION
except:
    raise Exception("Version %s or above of Pandas is required to run this notebook" % REQUIRED_MINIMUM_PANDAS_VERSION)

## Use decision optimization

### Passo 1: Importar a biblioteca

Execute o seguinte código para importar a biblioteca de Modelagem de Otimização de Decisão CPLEX. A biblioteca *DOcplex* contém os dois pacotes de modelagem: Programação Matemática (*docplex.mp*) e Programação por Restrições (*docplex.cp*).

In [2]:
import sys
try:
    import docplex.mp
except:
    raise Exception('Please install docplex. See https://pypi.org/project/docplex/')

### Passo 2: Modelar os dados

Os dados de entrada consistem em várias tabelas:

* **Departamentos**: lista todos os departamentos envolvidos na atribuição.
* **Habilidades**: lista todas as habilidades necessárias.
* **Turnos**: lista todos os turnos que precisam ser preenchidos, cada um contendo informações sobre o departamento, o dia da semana e os horários de início e término.
* **Enfermeiros**: lista todos os enfermeiros, identificados por seus nomes.
* **Habilidades dos Enfermeiros**: indica as habilidades de cada enfermeiro.
* **Requisitos de Habilidades**: lista o número mínimo de pessoas necessárias para cada departamento e habilidade.
* **Férias dos Enfermeiros**: lista os dias de folga de cada enfermeiro.
* **Associações de Enfermeiros**: lista os pares de enfermeiros que desejam trabalhar juntos.
* **Incompatibilidades de Enfermeiros**: lista os pares de enfermeiros que não desejam trabalhar juntos.

#### Carregando dados do Excel com pandas

Carregamos os dados de um arquivo Excel usando *pandas*.  
Cada planilha é lida em um *DataFrame* separado do *pandas*.

In [3]:
CSS = """
body {
    margin: 0;
    font-family: Helvetica;
}
table.dataframe {
    border-collapse: collapse;
    border: none;
}
table.dataframe tr {
    border: none;
}
table.dataframe td, table.dataframe th {
    margin: 0;
    border: 1px solid white;
    padding-left: 0.25em;
    padding-right: 0.25em;
}
table.dataframe th:not(:empty) {
    background-color: #fec;
    text-align: left;
    font-weight: normal;
}
table.dataframe tr:nth-child(2) th:empty {
    border-left: none;
    border-right: 1px dashed #888;
}
table.dataframe td {
    border: 2px solid #ccf;
    background-color: #f4f4ff;
}
    table.dataframe thead th:first-child {
        display: none;
    }
    table.dataframe tbody th {
        display: none;
    }
"""

In [4]:
from IPython.core.display import HTML
HTML('<style>{}</style>'.format(CSS))

from IPython.display import display

In [5]:
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO

In [6]:
# This notebook requires pandas to work
import pandas as pd
from pandas import DataFrame

# Make sure that xlrd package, which is a pandas optional dependency, is installed
# This package is required for Excel I/O
try:
    import xlrd
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install xlrd 
    else:
        !pip install --user xlrd      

In [7]:
# Use pandas to read the file, one tab for each table.
data_url = "https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/mp/jupyter/nurses_data.xls?raw=true"
nurse_xls_file = pd.ExcelFile(data_url)

df_skills = nurse_xls_file.parse('Skills')
df_depts = nurse_xls_file.parse('Departments')
df_shifts = nurse_xls_file.parse('Shifts')
# Rename df_shifts index
df_shifts.index.name = 'shiftId'

# Index is column 0: name
df_nurses = nurse_xls_file.parse('Nurses', header=0, index_col=0)
df_nurse_skills = nurse_xls_file.parse('NurseSkills')
df_vacations = nurse_xls_file.parse('NurseVacations')
df_associations = nurse_xls_file.parse('NurseAssociations')
df_incompatibilities = nurse_xls_file.parse('NurseIncompatibilities')

# Display the nurses dataframe
print("#nurses = {}".format(len(df_nurses)))
print("#shifts = {}".format(len(df_shifts)))
print("#vacations = {}".format(len(df_vacations)))

#nurses = 32
#shifts = 42
#vacations = 59


Além disso, introduzimos alguns dados globais adicionais:

* O tempo máximo de trabalho para cada enfermeiro.
* O número máximo e mínimo de turnos trabalhados por um enfermeiro em uma semana.

In [8]:
# maximum work time (in hours)
max_work_time = 40

# maximum number of shifts worked in a week.
max_nb_shifts = 5

Os turnos são armazenados em um *DataFrame* separado.

In [9]:
df_shifts

,department,day,start_time,end_time,min_req,max_req
shiftId,,,,,,
0,Emergency,Monday,2,8,3,5
1,Emergency,Monday,8,12,4,7
2,Emergency,Monday,12,18,2,5
3,Emergency,Monday,18,2,3,7
4,Consultation,Monday,8,12,10,13
5,Consultation,Monday,12,18,8,12
6,Cardiac Care,Monday,8,12,10,13
7,Cardiac Care,Monday,12,18,8,12
8,Emergency,Tuesday,2,8,3,5


### Passo 3: Preparar os dados

Precisamos pré-computar dados adicionais para os turnos.  
Para cada turno, precisamos do horário de início e de término expressos em horas, contando a partir do início da semana: segunda-feira às 8h é convertido para 8, terça-feira às 8h é convertido para 24 + 8 = 32, e assim por diante.

#### Sub-passo #1
Começamos adicionando uma coluna extra chamada `dow` (dia da semana), que converte a string "dia" em um inteiro de 0 a 6 (segunda-feira é 0, domingo é 6).

In [10]:
days = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
day_of_weeks = dict(zip(days, range(7)))

# utility to convert a day string e.g. "Monday" to an integer in 0..6
def day_to_day_of_week(day):
    return day_of_weeks[day.strip().lower()]

# for each day name, we normalize it by stripping whitespace and converting it to lowercase
# " Monday" -> "monday"
df_shifts["dow"] = df_shifts.day.apply(day_to_day_of_week)
df_shifts

,department,day,start_time,end_time,min_req,max_req,dow
shiftId,,,,,,,
0,Emergency,Monday,2,8,3,5,0
1,Emergency,Monday,8,12,4,7,0
2,Emergency,Monday,12,18,2,5,0
3,Emergency,Monday,18,2,3,7,0
4,Consultation,Monday,8,12,10,13,0
5,Consultation,Monday,12,18,8,12,0
6,Cardiac Care,Monday,8,12,10,13,0
7,Cardiac Care,Monday,12,18,8,12,0
8,Emergency,Tuesday,2,8,3,5,1


#### Sub-passo #2: Calcular o horário de início absoluto de cada turno.

Calcular o horário de início na semana é simples: basta adicionar `24 * dow` à coluna `start_time`. O resultado é armazenado em uma nova coluna chamada `wstart`.

In [11]:
df_shifts["wstart"] = df_shifts.start_time + 24 * df_shifts.dow

#### Sub-passo #3: Calcular o horário de término absoluto de cada turno.

Calcular o horário de término absoluto é um pouco mais complicado, pois alguns turnos se estendem até a meia-noite. Por exemplo, o Turno #3 começa na segunda-feira às 18:00 e termina na terça-feira às 2:00. O horário de término absoluto do Turno #3 é 26, não 2.  
A regra geral para calcular o horário de término absoluto é:

`abs_end_time = end_time + 24 * dow + (start_time >= end_time ? 24 : 0)`

Novamente, usamos *pandas* para adicionar uma nova coluna calculada chamada `wend`. Isso é feito usando o método `apply` do *pandas* com uma função anônima `lambda` aplicada às linhas. O parâmetro `raw=True` impede a criação de uma *Series* do *pandas* para cada linha, o que melhora significativamente o desempenho em conjuntos de dados grandes.

In [12]:
# an auxiliary function to calculate absolute end time of a shift
def calculate_absolute_endtime(start, end, dow):
    return 24*dow + end + (24 if start>=end else 0)

# store the results in a new column
df_shifts["wend"] = df_shifts.apply(lambda row: calculate_absolute_endtime(
        row.start_time, row.end_time, row.dow), axis=1)

#### Sub-passo #4: Calcular a duração de cada turno.

Calcular a duração de cada turno agora é uma diferença direta entre colunas. O resultado é armazenado na coluna `duration`.

In [13]:
df_shifts["duration"] = df_shifts.wend - df_shifts.wstart

#### Sub-passo #5: Calcular a demanda mínima para cada turno.

A demanda mínima é o produto da duração (em horas) pelo número mínimo de enfermeiros requeridos. Assim, em termos de horas de enfermeiro, essa demanda é armazenada em outra nova coluna chamada `min_demand`.

Por fim, exibimos o *DataFrame* de turnos atualizado com todas as colunas calculadas.

In [14]:
# also compute minimum demand in nurse-hours
df_shifts["min_demand"] = df_shifts.min_req * df_shifts.duration
# finally check the modified shifts dataframe
df_shifts

,department,day,start_time,end_time,min_req,max_req,dow,wstart,wend,duration,min_demand
shiftId,,,,,,,,,,,
0,Emergency,Monday,2,8,3,5,0,2,8,6,18
1,Emergency,Monday,8,12,4,7,0,8,12,4,16
2,Emergency,Monday,12,18,2,5,0,12,18,6,12
3,Emergency,Monday,18,2,3,7,0,18,26,8,24
4,Consultation,Monday,8,12,10,13,0,8,12,4,40
5,Consultation,Monday,12,18,8,12,0,12,18,6,48
6,Cardiac Care,Monday,8,12,10,13,0,8,12,4,40
7,Cardiac Care,Monday,12,18,8,12,0,12,18,6,48
8,Emergency,Tuesday,2,8,3,5,1,26,32,6,18


### Passo 4: Configurar o modelo

In [15]:
from pulp import LpProblem, LpVariable, lpSum, LpStatus, LpStatusOptimal
import pulp

# Criando o problema de otimização
prob = pulp.LpProblem("nurses", pulp.LpMaximize)

# Definindo as coleções de enfermeiros e turnos
all_nurses = df_nurses.index.values  # Lista de enfermeiros
all_shifts = df_shifts.index.values  # Lista de turnos

# Definindo as variáveis binárias de atribuição
assigned = pulp.LpVariable.dicts(
    "assign", 
    [(nurse, shift) for nurse in all_nurses for shift in all_shifts], 
    cat='Binary'
)

In [16]:
import pandasql as psql
import sqlite3

df = pd.DataFrame(df_shifts)

# Conectar ao banco de dados SQLite na memória
conn = sqlite3.connect(':memory:')

# Carregar o DataFrame em uma tabela SQL temporária
df.to_sql('pessoas', conn, index=False, if_exists='replace')

# Consulta SQL para encontrar turnos que ocorrem no mesmo horário
query = """
SELECT group_concat(shiftId) as turnos_simultaneos
FROM df
GROUP BY wstart 
HAVING COUNT(*) > 1
"""

# Executar a consulta no DataFrame
result = psql.sqldf(query, locals())

# Exibir o resultado
print(result)

   turnos_simultaneos
0               1,4,6
1               2,5,7
2             9,12,14
3            10,13,15
4               11,16
5               18,21
6               19,22
7               24,27
8               25,28
9               30,33
10              31,34


In [ ]:
#Adicionar a restrição para cada enfermeiro e grupo de turnos simultâneos
for nurse in all_nurses:
    for _, row in result.iterrows():
        # Pegar os turnos simultâneos e transformá-los em uma lista de inteiros
        simultaneous_shifts = [int(shift_id) for shift_id in row['turnos_simultaneos'].split(',')]
        
        # Criar a restrição: a soma das variáveis de decisão desses turnos deve ser <= 1 para cada enfermeiro
        prob += lpSum([assigned[(nurse, shift)] for shift in simultaneous_shifts]) <= 1, f"RestrSimult_{nurse}_{simultaneous_shifts}"


In [28]:
assigned[('Anne',1)] + assigned[('Anne',4)] + assigned[('Anne',4)] <= 1

1*assign_('Anne',_np.int64(1)) + 2*assign_('Anne',_np.int64(4)) + 0